## This is aim to visualize the crop progress in the Urbana and other town
- Combined with the weather data and soil data. 
- Interploate the weather data to create the CGDD,CGDD in the first and last three weeks. Also for CRPT. 
- Merged with soil data(for city with multiple soil type(get the mean). Every time run the Data creation function,should use the Soil check.ipnb to update the soil dictionary!
  
- some issue with the names of the city and soil are not the same in UIUC(FIRST) and Weather(Soil_dic)


### file needed
- 2011_2018_ALL_FIRST_DATA.csv,2011_2018_UIUC_ALL_DATA.csv
- Soil_dic_added.csv(obtained by running the Soil_check.ipynb)
- InterpolatedParameters.csv(obtianed by the running 

In [306]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import os
import math
import re
import time
import pandas as pd
import numpy as np
import os
import warnings
import seaborn
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
from datetime import datetime, timedelta
import seaborn as sns, numpy as np
from tqdm import tqdm
import warnings; warnings.simplefilter('ignore')


In [307]:
#define the saving path
saving_path = "town_csv"
try:
    os.stat(saving_path)
except:
    os.mkdir(saving_path)       


In [308]:

def NA_count(arg1):
    col_list = arg1.columns
    i=0
    df = pd.DataFrame(columns = ["var", "missing_percent"])
    for col in col_list:
        
        a = len(arg1[col][pd.isnull(arg1[col])])/len(arg1)
        df.loc[i,"var"]= col
        df.loc[i,"missing_percent"]= a*100
        i= i+1
    return(df)

In [309]:
# read in the data for weather and both 
file_path= 'data'
filecsv_list = []
for root, dirs , files in os.walk(file_path):
    for file in files:
        if os.path.splitext(file)[1] == '.csv':
            filecsv_list.append(file)
filecsv_list

['2011_2018_ALL_FIRST_DATA.csv',
 '2011_2018_ALL_FIRST_DATA_edited.csv',
 '2011_2018_ALL_FIRST_DATA_ori.csv',
 '2011_2018_UIUC_ALL_DATA.csv',
 'Farms_Soils_Year_LL.csv',
 'FieldSiteLocations.csv',
 'InterpolatedParameters.csv',
 'Soil_dic.csv',
 'soil_dic_6L.csv',
 'Soil_dic_added.csv',
 'Soil_dic_added_6L.csv',
 'soil_dic_interpolated.csv',
 'standardized_soil_fixed.csv',
 'weather_20190226.csv']

In [310]:
Soil_city = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*Farms.*', f)][0]))
Soil_dic_6L = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*6L.*', f)][0]))
Soil_dic_6L.drop('Unnamed: 0', axis=1, inplace=True)
soil_dic_interpolated = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*dic.*', f)][0]))
soil_dic_interpolated.drop('Unnamed: 0', axis=1, inplace=True)
soil_dic_added = pd.read_csv(os.path.join(file_path, 'Soil_dic_added_6L.csv'))

which is interesting that the soil data is unique for each. Direct match the FIRST or UIUC data with the soil data.

In [311]:
soil_dic_added.tail()

,1_% Total Clay < 0.002,1_%Total Silt 0.002-0.05,1_% Total Sand 0.05-2,2_% Total Clay < 0.002,2_%Total Silt 0.002-0.05,2_% Total Sand 0.05-2,3_% Total Clay < 0.002,3_%Total Silt 0.002-0.05,3_% Total Sand 0.05-2,4_% Total Clay < 0.002,4_%Total Silt 0.002-0.05,4_% Total Sand 0.05-2,5_% Total Clay < 0.002,5_%Total Silt 0.002-0.05,5_% Total Sand 0.05-2,6_% Total Clay < 0.002,6_%Total Silt 0.002-0.05,6_% Total Sand 0.05-2,Num_of_Layers,Soil_Name_lower
285,27.05,68.10,3.85,27.05,68.1,3.85,29.200000,65.95,3.500000,30.42,64.050,3.730,33.600000,62.250000,3.050000,42.00,54.000000,1.000000,7.00,Ipava/Tama silty clay loam
286,46.85,40.00,7.55,46.85,40.0,7.55,46.850000,37.80,9.150000,42.74,39.325,11.345,39.110000,42.860000,11.580000,35.95,46.300000,11.500000,7.00,Osco/Durand silt loam
287,22.75,58.00,18.25,22.75,58.2,18.05,27.633333,66.00,5.366667,29.00,65.200,4.600,30.733333,62.966667,4.466667,32.50,61.033333,4.966667,6.75,Drummer/Dana/Parr/Raub silty clay loam
288,21.25,65.25,12.00,20.50,66.9,10.75,21.200000,67.15,9.650000,25.40,65.400,7.700,33.700000,60.850000,4.400000,37.15,59.350000,2.500000,10.00,Elburn/Flanagan silty clay loam
289,23.45,66.25,9.80,22.70,67.9,8.55,19.400000,65.30,13.300000,22.80,64.800,11.400,30.400000,62.000000,6.600000,36.30,59.700000,3.000000,7.50,Flanagan/Catlin silt loam


In [312]:
os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*UIUC.*', f)][0])
UIUC_2011_2017_ALL_DATA= pd.read_csv(os.path.join(file_path,'2011_2018_UIUC_ALL_DATA.csv'),encoding = "latin-1")

In [313]:
Soil_list = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*fixed.*', f)][0]),encoding = "latin-1")
# UIUC_2011_2017_ALL_DATA= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*UIUC.*', f)][0]),encoding = "latin-1")
UIUC_2011_2017_ALL_DATA['Location Town'].value_counts()
#

Monmouth      830
New Berlin    830
Mt. Morris    817
Goodfield     798
DeKalb        617
Urbana        616
Erie          616
Perry         556
St. Peter     501
Dwight        475
Belleville    416
Elkville      236
Fenton        201
Name: Location Town, dtype: int64

In [314]:
uiuc_soil_list= UIUC_2011_2017_ALL_DATA['Soil Type'].value_counts().index.values

In [315]:
Weather_interp= pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*InterpolatedParameters.*', f)][0]))
Weather_interp.head()
# Weather_interp['CITY'].value_counts()

,Unnamed: 0,CITY,DATE,Interpolated TMIN,Interpolated TMAX,Interpolated PRCP
0,0,F_BELLEVILLE,1/1/2011,27.727507,59.475263,0.041975
1,1,F_BELLEVILLE,1/2/2011,18.079533,33.972957,0.000000
2,2,F_BELLEVILLE,1/3/2011,22.292492,44.159572,0.000000
3,3,F_BELLEVILLE,1/4/2011,23.356365,42.484850,0.000000
4,4,F_BELLEVILLE,1/5/2011,20.680546,40.737094,0.000000


In [316]:
Weather_interp.tail(5)

,Unnamed: 0,CITY,DATE,Interpolated TMIN,Interpolated TMAX,Interpolated PRCP
146095,146095,U_ST. PETER_17,2/9/2017,14.558895,34.894290,0.003719
146096,146096,U_ST. PETER_17,3/11/2017,27.490374,42.481815,0.000000
146097,146097,U_ST. PETER_17,3/12/2017,23.881506,35.738056,0.000000
146098,146098,U_ST. PETER_17,3/14/2017,23.401869,32.677389,0.113543
146099,146099,U_ST. PETER_17,12/24/2017,27.868662,35.875802,0.202288


In [317]:
Weather_interp.columns

Index(['Unnamed: 0', 'CITY', 'DATE', 'Interpolated TMIN', 'Interpolated TMAX',
       'Interpolated PRCP'],
      dtype='object')

In [318]:

First_11_17_all= pd.read_csv(os.path.join(file_path,'2011_2018_ALL_FIRST_DATA.csv'))
First_11_17_all.columns

Index(['DataSource', 'Company', 'Product', 'Technology', 'RM', 'Yield (Bu/A)',
       'Moisture %', 'Lodging %', 'Stand (x1000)', 'Gross $/A', 'Income Rank',
       'Year', 'Location_Town', 'Location_County', 'Location State', 'Season',
       'Prev_Crop', 'Previous_Pest', 'Previous Yield ()',
       'Previous Yield ().1', 'Soil_Type', 'Soil_slope', 'Soil_drained',
       'Soil_Irr', 'Soil condition', 'Soil_P', 'Soil_K', 'Soil_pH', 'Soil_OM%',
       'Soil_CEC', 'PestMang', 'FertN (units)', 'FertP (units)',
       'FertK (units)', 'Seed_date', 'Seed rate ()', 'Seed space (in)',
       'Harvest_date', 'Harvest rate ()'],
      dtype='object')

In [319]:
First_11_17_all['Location_Town'].unique()

array(['FORSYTH', 'thomasboro', 'TOLUCA', 'TOWANDA', 'WATSEKA', 'TUSCOLA',
       'GRAND RIDGE', 'SUBLETTE', 'WALNUT', 'MAZON', 'WINNEBAGO',
       'BELLEVILLE', 'RAMSEY', 'MARION', 'FLORA', 'SALEM', 'EFFINGHAM',
       'WILLIAMSVILLE', 'virden', 'DELAVAN', 'GALVA', 'MACOMB',
       'JACKSONVILLE', 'PEARL CITY', 'MILLEDGEVILLE', 'WOODSTOCK',
       'Malta', 'Virden', 'Grand Ridge', 'Flora', 'Belleville', 'Watseka',
       'Sublette', 'Tuscola', 'Milledgeville', 'Williamsville', 'Bethany',
       'Macomb', 'Towanda', 'Clayton', 'Paxton', 'Vandalia', 'Mazon',
       'Woodstock', 'Delavan', 'Winnebago', 'Shumway', 'Forsyth',
       'Walnut', 'Warren', 'Salem', 'Greenville', 'Du Quoin', 'Galva',
       'Rossville', 'Geneseo', 'Moweaqua', 'Thomasboro', 'Pearl City'],
      dtype=object)

In [320]:
First_11_17_all['Location_Town'].isna().mean()

0.0

In [321]:
Soil_dic = pd.read_csv(os.path.join(file_path,'Soil_dic.csv'),encoding = "latin-1")

In [322]:
# Categorical boolean mask
categorical_feature_mask = UIUC_2011_2017_ALL_DATA.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = UIUC_2011_2017_ALL_DATA.columns[categorical_feature_mask].tolist()
categorical_cols[8:10]

['Previous Crop', 'Previous Pest']

In [323]:

n = sum(UIUC_2011_2017_ALL_DATA[categorical_cols[0]].value_counts())

c_list = []
for i in range(len(categorical_cols)):
    n0 =  sum(UIUC_2011_2017_ALL_DATA[categorical_cols[i]].value_counts())
    c_list.append(n0/n
                 )
c_list.sort()
pd.DataFrame(c_list).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.936309,0.987143,0.987143,0.987143,0.987143,1.0,1.0,1.0,1.0,1.0,1.109206,1.10965,1.10965,1.10965,1.10965


In [324]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
tmp =  UIUC_2011_2017_ALL_DATA[categorical_cols[8:10]].copy()
tmp =tmp.fillna("Missing")
UIUC_2011_2017_ALL_DATA[categorical_cols[8:10]] = tmp.apply(lambda col: le.fit_transform(col))

In [325]:
#First 
# Categorical boolean mask
categorical_feature_mask = First_11_17_all.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = First_11_17_all.columns[categorical_feature_mask].tolist()
categorical_cols[8:10]
n = sum(First_11_17_all[categorical_cols[0]].value_counts())

c_list = []
for i in range(len(categorical_cols)):
    n0 =  sum(First_11_17_all[categorical_cols[i]].value_counts())
    c_list.append(n0/n
                 )
c_list.sort()
pd.DataFrame(c_list).T
# acceptable missing 

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.339443,0.837312,0.874822,0.888036,0.95638,0.956451,0.956451,0.956451,0.960642,0.969309,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [326]:
# Categorical boolean mask
categorical_feature_mask =First_11_17_all.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols =First_11_17_all.columns[categorical_feature_mask].tolist()

le = LabelEncoder()
tmp =  First_11_17_all[categorical_cols[8:10]].copy()
tmp =tmp.fillna("Missing")
First_11_17_all[categorical_cols[8:10]] = tmp.apply(lambda col: le.fit_transform(col))

In [327]:
# set the colname the same
First_11_17_all.columns  = UIUC_2011_2017_ALL_DATA.columns
First_11_17_all.columns ==UIUC_2011_2017_ALL_DATA.columns
#len(First_11_17_all.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

__The columns len and rough contents matched for uiuc and first data sets. Reset the column names of both sets according to the uiuc sets.__

In [328]:
UIUC_2011_2017_ALL_DATA.head()

,DataSource,Company,Product,Technology,RM,Yield (Bu/A),Moisture %,Lodging %,Stand (x1000),Gross $/A,...,Soil_CEC,PestMang,FertN (units),FertP (units),FertK (units),Seed Date,Seed Rate (/A),Seed Space (in),Harvest Date,Harvest Rate (/A)
0,UIUC,AgriGold,A6442STXRIB,GMO,107.0,223.0,14.9,0.0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
1,UIUC,AgriGold,A6472VT3PRIB,GMO,109.0,230.0,15.7,0.0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
2,UIUC,AgriGold,A6499STXRIB,GMO,110.0,248.0,18.3,0.0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
3,UIUC,AgriGold,A6533VT3PRIB,GMO,112.0,249.0,17.3,0.0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN
4,UIUC,AgriGold,A6559STXRIB,GMO,113.0,247.0,16.8,0.0,NaN,NaN,...,NaN,"PPI-BicepIIMagnum, POST-Impact",170.0,NaN,NaN,5/5/2014,34000.0,NaN,10/1/2014,NaN


In [329]:
UIUC_2011_2017_ALL_DATA.groupby('Year')['Moisture %'].describe()

,count,mean,std,min,25%,50%,75%,max
Year,,,,,,,,
2011,1134.0,20.797795,3.765813,0.0,17.825,21.4,23.575,30.6
2012,628.0,18.540287,3.284182,12.8,16.000,17.8,21.000,30.0
2013,1123.0,22.322262,3.061640,15.5,19.900,22.0,24.500,31.6
2014,1177.0,19.399320,3.170884,13.7,17.100,18.8,21.300,32.0
2015,986.0,16.410041,2.744444,11.9,14.500,15.7,17.400,26.8
2016,827.0,18.334825,2.839917,13.0,16.400,17.9,20.050,27.5
2017,849.0,20.539694,3.092259,12.7,18.800,20.3,22.300,31.1
2018,742.0,17.415768,2.166530,13.0,15.700,17.4,18.900,25.4


To avoid too much missing, drop those with 10% missing var

In [330]:
tmp = NA_count(UIUC_2011_2017_ALL_DATA)
UIUC_2011_2017_ALL_DATA = UIUC_2011_2017_ALL_DATA.loc[:,(tmp['missing_percent']<30).values]
tmp 

,var,missing_percent
0,DataSource,9.88148
1,Company,0
2,Product,0.0399521
3,Technology,0
4,RM,0.0399521
5,Yield (Bu/A),0.585964
6,Moisture %,0.572646
7,Lodging %,5.51338
8,Stand (x1000),100
9,Gross $/A,100


In [331]:
NA_count(First_11_17_all)

,var,missing_percent
0,DataSource,0
1,Company,4.35493
2,Product,4.36203
3,Technology,4.35493
4,RM,4.39756
5,Yield (Bu/A),4.35493
6,Moisture %,4.35493
7,Lodging %,4.35493
8,Stand (x1000),4.35493
9,Gross $/A,4.35493


In [332]:
UIUC_2011_2017_ALL_DATA.groupby('Location Town')['Seed Date'].value_counts()

Location Town  Seed Date
Belleville     5/5/2015      81
               5/5/2014      71
               4/25/2013     70
               4/18/2016     69
               5/18/2017     59
DeKalb         5/7/2014     121
               5/8/2013     108
               5/8/2017     103
               5/8/2011     101
               4/24/2012     97
               4/23/2016     87
Dwight         5/5/2011     118
               5/6/2014      96
               4/19/2017     86
               4/19/2016     79
               4/24/2014      8
Elkville       5/18/2011     85
               5/1/2015      81
               4/25/2013     70
Erie           5/7/2014     121
               5/8/2013     108
               4/23/2015    102
               5/3/2011     101
               4/24/2012     97
Fenton         5/8/2017     103
Goodfield      5/6/2011     118
               4/17/2015    110
               5/8/2013     110
               5/8/2014     104
               4/18/2012    103
               

In [333]:
First_11_17_all['Seed Date'][0]

'4/26/2018'

In [334]:
First_11_17_all.groupby('Location Town')['Seed Date'].value_counts()

Location Town  Seed Date
BELLEVILLE     5/11/2018    62
Belleville     5/15/2012    62
               5/26/2013    62
               5/26/2017    62
               6/11/2015    62
               6/2/2011     62
               6/2/2016     62
               5/10/2014    31
Bethany        4/26/2014    93
               4/29/2015    62
               5/16/2013    62
               5/18/2011    61
               4/27/2012    60
Clayton        5/12/2011    62
               5/16/2012    62
               5/19/2015    62
               5/23/2016    62
               5/24/2013    62
               5/8/2014     62
DELAVAN        4/26/2018    62
Delavan        5/10/2012    62
               5/15/2016    62
               5/16/2017    62
               5/6/2014     62
               5/6/2011     61
               5/14/2015    60
               5/15/2013    59
Du Quoin       6/1/2011     62
               6/9/2016     62
               4/25/2017    61
                            ..
Watseka       

In [335]:
UIUC_2011_2017_ALL_DATA.groupby('Location Town')['Soil Type'].value_counts()

Location Town  Soil Type               
Belleville     Ebbert silt loam            291
               Caseyville silt loam         59
DeKalb         Flanagan silty clay loam    514
               Elpaso silty clay loam      103
Dwight         Reddick silty clay loam     387
Elkville       Okaw silt loam              236
Erie           Beaucoup silty clay loam    616
Fenton         Coffeen silt loam           103
Goodfield      Ipava silt loam             710
Monmouth       Sable silty clay loam       755
Mt. Morris     Muscatine silt loam         719
New Berlin     Sable silt loam             755
Perry          Herrick silt loam           556
St. Peter      Bluford silt loam           376
               Darmstadt silt loam          59
Urbana         Flanagan silt loam          528
Name: Soil Type, dtype: int64

In [336]:
Soil_list.head()

,Num_of_Layers,Soil Name,Soil Desc,Pedon No.,1_Layer,1_Horizon,1_Orig Hzn,1_Depth (cm),1_low,1_up,...,30_Water Content 33 kPa,30_Water Content 1500 kPa,30_% OC (WB),30_NH4OAC Extractable Bases Ca,30_NH4OAC Extractable Bases Mg,30_NH4OAC Extractable Bases Na,30_NH4OAC Extractable Bases K,30_NH4OAC Extractable Bases Sum Bases,30_CEC8 Sum Cats,30_CEC7 NH4 OAC
0,5,Mokena,NaN,NaN,NaN,H1,NaN,0-13,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,Myrtle,NaN,NaN,NaN,A,NaN,0-20,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,Gilford,"Coarse-loamy, mixed, superactive, mesic Typic ...",75IL103004,IL24067,Ap,Ap,0-18,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Parr,"A Fine-loamy, mixed, active, mesic Oxyaquic Ar...",78IL103015,IL25822,Ap,Ap,0-18,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Raub,"Fine-silty, mixed, superactive, mesic Aquic Ar...",61IL019002,IL18852,A1,A1,0-20,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
Soil_city = pd.read_csv(os.path.join(file_path,[f for f in filecsv_list if re.match(r'.*Farms.*', f)][0]))

In [338]:
Soil_city.head()


,Study,Start,Stop,City,County,C_Lat,C_Lon,Y11,Y12,Y13,Y14,Y15,Y16,Y17,StudyCity,Unnamed: 15,SimpleCity,SimpleLL,Soil
0,FIRST,2011.0,2017.0,Belleville,St. Clair,38.513,-89.984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FIRST-Belleville,NaN,Belleville,F_Belleville,Muscatune silt loam
1,UIUC,2011.0,2016.0,Belleville_11,St. Clair,38.519,-89.842,NaN,NaN,NaN,NaN,NaN,NaN,X,UIUC-Belleville_11,NaN,Belleville,U_Belleville_11,Ebbert silt loam
2,UIUC,2017.0,2017.0,Belleville_17,St. Clair,38.535,-89.899,X,X,X,X,X,X,NaN,UIUC-Belleville_17,NaN,Belleville,U_Belleville_17,Ebbert silt loam
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Belleville Count,3,NaN,NaN
4,FIRST,2011.0,2015.0,Bethany,Moultrie,39.645,-88.741,NaN,NaN,NaN,NaN,NaN,X,X,FIRST-Bethany,NaN,Bethany,F_Bethany,Drummer/Flanagan


In [339]:
ind = UIUC_2011_2017_ALL_DATA["Seed Date"].isna()
UIUC_2011_2017_ALL_DATA = UIUC_2011_2017_ALL_DATA.loc[-ind,:]
ind = UIUC_2011_2017_ALL_DATA["Harvest Date"].isna()
UIUC_2011_2017_ALL_DATA = UIUC_2011_2017_ALL_DATA.loc[-ind,:]

In [340]:
ind = First_11_17_all["Seed Date"].isna()
First_11_17_all = First_11_17_all.loc[-ind,:]
ind = First_11_17_all["Harvest Date"].isna()
First_11_17_all = First_11_17_all.loc[-ind,:]

In [341]:
 Weather_interp.loc[:,'DATE'].tail()

146095      2/9/2017
146096     3/11/2017
146097     3/12/2017
146098     3/14/2017
146099    12/24/2017
Name: DATE, dtype: object

In [342]:
# check the city name in First and UIUC match in soil or weather set 

In [343]:
city_list_weather = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()).unique()
print(city_list_weather)

['Belleville' 'Bethany' 'Clayton' 'Delavan' 'Du quoin' 'Flora' 'Forsyth'
 'Galva' 'Geneseo' 'Grand ridge' 'Greenville' 'Macomb' 'Malta' 'Mazon'
 'Milledgeville' 'Moweaqua' 'Paxton' 'Pearl city' 'Rossville' 'Salem'
 'Shumway' 'Sublette' 'Thomasboro' 'Towanda' 'Tuscola' 'Vandalia' 'Virden'
 'Walnut' 'Warren' 'Watseka' 'Williamsville' 'Winnebago' 'Woodstock'
 'Dwight' 'Elkville' 'Erie' 'Fenton' 'Goodfield' 'Monmoth' 'Mt. morris'
 'New berlin' 'Perry' 'Urbana' 'Dekalb' 'St. peter']


In [344]:
city_uiuc = UIUC_2011_2017_ALL_DATA['Location Town'].unique()

In [345]:
city_first= First_11_17_all["Location Town"].unique()

In [346]:
city_first

array(['FORSYTH', 'thomasboro', 'TOLUCA', 'TOWANDA', 'WATSEKA', 'TUSCOLA',
       'GRAND RIDGE', 'SUBLETTE', 'MAZON', 'WINNEBAGO', 'BELLEVILLE',
       'RAMSEY', 'MARION', 'FLORA', 'SALEM', 'EFFINGHAM', 'WILLIAMSVILLE',
       'virden', 'DELAVAN', 'GALVA', 'MACOMB', 'JACKSONVILLE',
       'PEARL CITY', 'MILLEDGEVILLE', 'WOODSTOCK', 'Malta', 'Virden',
       'Grand Ridge', 'Flora', 'Belleville', 'Watseka', 'Sublette',
       'Tuscola', 'Milledgeville', 'Williamsville', 'Bethany', 'Macomb',
       'Towanda', 'Clayton', 'Paxton', 'Vandalia', 'Mazon', 'Woodstock',
       'Delavan', 'Winnebago', 'Shumway', 'Forsyth', 'Walnut', 'Warren',
       'Salem', 'Greenville', 'Du Quoin', 'Galva', 'Rossville', 'Geneseo',
       'Moweaqua', 'Thomasboro', 'Pearl City'], dtype=object)

In [347]:
first_not_in_list = []
for city in city_first:
    if city.lower().capitalize() not in city_list_weather:
        first_not_in_list.append(city.lower().capitalize())
print(first_not_in_list)

['Toluca', 'Ramsey', 'Marion', 'Effingham', 'Jacksonville']


In [348]:
uiuc_not_in_list = []
for city in city_uiuc:
    if city.lower().capitalize() not in city_list_weather:
        uiuc_not_in_list.append(city.lower().capitalize())
print(uiuc_not_in_list)

['Monmouth']


In [349]:
UIUC_2011_2017_ALL_DATA.loc[UIUC_2011_2017_ALL_DATA['Location Town']== 'Monmouth',:].shape

(755, 22)

In [350]:




def get_df_UIUC(Urbana):
    
    def city_correct(Urbana):
        if Urbana == 'Monmouth':
            return('Monmoth')
        else:return(Urbana)
    def sp_correct(soil):
        if soil == "muscatine": return("muscatune")
        elif soil == "darmstadt":return("darmstat")
        elif soil == "reddick" : return("redick")
        elif soil ==  "caseyville" :return('ebbert')
        else: return(soil)
        
    Urbana_df = UIUC_2011_2017_ALL_DATA.loc[(UIUC_2011_2017_ALL_DATA['Location Town']== Urbana),:]
    Num_col = list(Urbana_df.select_dtypes([np.number]).columns.values)
    Num_col.extend(['Seed Date','Harvest Date','Soil Type'])
    Urbana_df = Urbana_df.loc[:,Num_col]   
    
    def select_name(name):
        if len(name.split("/"))>1:return(name.split("/")[0].split()[0].lower())
        else:
            return(name.split()[0].lower())
       
    if  isinstance(Urbana_df['Soil Type'].unique()[0], str):
        Urbana_df['Soil Type'] = Urbana_df['Soil Type'].fillna(Urbana_df['Soil Type'].unique()[0])
    else:Urbana_df['Soil Type'] = Urbana_df['Soil Type'].fillna(Urbana_df['Soil Type'].unique()[1])
        
    Urbana_df['Soil_Name_lower'] = Urbana_df['Soil Type'].apply(lambda x: select_name(x))
    Urbana_df['Soil_Name_lower'] = Urbana_df['Soil_Name_lower'].apply(lambda x: sp_correct(x))

        
    Urbana_df = pd.merge(Urbana_df, soil_dic_added, how='left',on = 'Soil_Name_lower')

    def get_temperature(CITY):
        ind = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()) == CITY.lower().capitalize()

        temperature =Weather_interp.loc[ind,:]

        temperature = temperature[['DATE','CITY','Interpolated TMIN','Interpolated TMAX']]

        temperature = temperature.dropna()
        temperature = temperature.drop_duplicates()
        temperature.columns = ['DATE', 'CITY', 'TMIN', 'TMAX']
        temperature.head()

        upper_index = (temperature[temperature['TMAX'] > 86].index.values.tolist())
        lower_index_tmin = (temperature[temperature['TMIN'] < 50].index.values.tolist())
        lower_index_tmax = (temperature[temperature['TMAX'] < 50].index.values.tolist())
        temperature.loc[upper_index, 'TMAX'] = 86
        temperature.loc[lower_index_tmin, 'TMIN'] = 50
        temperature.loc[lower_index_tmax, 'TMAX'] = 50
        temperature ['GDD'] = temperature.mean(axis=1) - 50
        temperature['PRCP']  = Weather_interp.loc[ind,'Interpolated PRCP']
        return(temperature)


    # Urbana_df = Urbana_df.drop(['Soil Name', 'Soil Type'], axis=1)
    temperature = get_temperature(city_correct(Urbana)) 

    def  std_GDD (x):
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])
        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['GDD'].std()#
        return(tmp)

    def std_PRCP(x ):
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])
        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['PRCP'].std()#
        return(tmp)




    def row_cumulative_GDD(x,first = False ,last = False,length =21):
        if first == True:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[0])+timedelta(days=length)
        elif last == True:
            start_date=pd.to_datetime(x[1])-timedelta(days=length)
            end_date=pd.to_datetime(x[1])
        else:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[1])


        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['GDD'].sum()#
        return(tmp)

    def row_cumulative_PRCP(x,first = False ,last = False,length =21):

        if first == True:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[0])+timedelta(days=length)
        elif last == True:
            start_date=pd.to_datetime(x[1])-timedelta(days=length)
            end_date=pd.to_datetime(x[1])
        else:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[1])

        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['PRCP'].sum()#
        return(tmp)
     
    Urbana_df['CGDD'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_GDD,axis=1)
    Urbana_df['std_GDD'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_GDD,axis=1)
    Urbana_df['CPRCP']= Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)
    Urbana_df['std_PRCP']= Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)







    Urbana_df['CGDD_first21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,first = True),axis=1)
    Urbana_df['CGDD_last21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,last = True),axis=1)
    Urbana_df['CPRCP_first21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,first = True),axis=1)
    Urbana_df['CPRCP_last21'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,last = True),axis=1)
 #   tmp = NA_count(Urbana )
#    Urbana  = Urbana .loc[:,(tmp['missing_percent']<10).values]
    return(Urbana_df)
#['Product','Year','Yield (Bu/A)','Seed Date','Harvest Date','Moisture %', 'Seed Rate (/A)','Lodging %']
#Urbana  =Urbana[col_new]


#ind = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()) == CITY.lower().capitalize()

## Get the city list from UIUC

In [351]:
UIUC_city_list = UIUC_2011_2017_ALL_DATA['Location Town'].unique()
UIUC_city_list 

array(['Belleville', 'Erie', 'St. Peter', 'Perry', 'New Berlin', 'Dwight',
       'Urbana', 'Mt. Morris', 'Monmouth', 'Goodfield', 'DeKalb',
       'Elkville', 'Fenton'], dtype=object)

In [352]:
Weather_interp.head()

,Unnamed: 0,CITY,DATE,Interpolated TMIN,Interpolated TMAX,Interpolated PRCP
0,0,F_BELLEVILLE,1/1/2011,27.727507,59.475263,0.041975
1,1,F_BELLEVILLE,1/2/2011,18.079533,33.972957,0.000000
2,2,F_BELLEVILLE,1/3/2011,22.292492,44.159572,0.000000
3,3,F_BELLEVILLE,1/4/2011,23.356365,42.484850,0.000000
4,4,F_BELLEVILLE,1/5/2011,20.680546,40.737094,0.000000


In [255]:
get_temperature(Urbana) 

,DATE,CITY,TMIN,TMAX,GDD,PRCP


In [251]:
Urbana_df['CGDD'] =Urbana_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_GDD,axis=1)

In [253]:
Urbana_df.loc[:,['Seed Date', 'Harvest Date']]

,Seed Date,Harvest Date
0,4/23/2014,10/10/2014
1,4/23/2014,10/10/2014
2,4/23/2014,10/10/2014
3,4/23/2014,10/10/2014
4,4/23/2014,10/10/2014
5,4/23/2014,10/10/2014
6,4/18/2012,9/20/2012
7,4/18/2012,9/20/2012
8,4/30/2018,10/1/2018
9,4/18/2012,9/20/2012


In [354]:
# clean and merge the soil data for each city on the list  Belleville
with tqdm(total=len( UIUC_city_list)) as pbar:    
    for city in UIUC_city_list: 
        print(city)
        city_clean = city
        city_clean = city_clean.replace(' ','_')
        city_clean =city_clean.replace('._','_')
        vars()[city_clean]  = get_df_UIUC(city)
        print((eval(city_clean)).isna().mean())
        eval(city_clean).to_csv(os.path.join(saving_path,(city_clean+'.csv')))
        print(eval(city_clean).describe())
        pbar.update(1) 

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Belleville
RM                          0.000000
Yield (Bu/A)                0.022857
Moisture %                  0.022857
Lodging %                   0.000000
Year                        0.000000
Previous Crop               0.000000
Previous Pest               0.000000
FertN (units)               0.000000
Seed Rate (/A)              0.000000
Seed Date                   0.000000
Harvest Date                0.000000
Soil Type                   0.000000
Soil_Name_lower             0.000000
1_% Total Clay < 0.002      0.000000
1_%Total Silt 0.002-0.05    0.000000
1_% Total Sand 0.05-2       0.000000
2_% Total Clay < 0.002      0.000000
2_%Total Silt 0.002-0.05    0.000000
2_% Total Sand 0.05-2       0.000000
3_% Total Clay < 0.002      0.000000
3_%Total Silt 0.002-0.05    0.000000
3_% Total Sand 0.05-2       0.000000
4_% Total Clay < 0.002      0.000000
4_%Total Silt 0.002-0.05    0.000000
4_% Total Sand 0.05-2       0.000000
5_% Total Clay < 0.002      0.000000
5_%Total Silt 0.002-0.05   

  8%|██████▍                                                                            | 1/13 [00:18<03:43, 18.59s/it]

Erie
RM                          0.00189
Yield (Bu/A)                0.00000
Moisture %                  0.00000
Lodging %                   0.00000
Year                        0.00000
Previous Crop               0.00000
Previous Pest               0.00000
FertN (units)               0.00000
Seed Rate (/A)              0.00000
Seed Date                   0.00000
Harvest Date                0.00000
Soil Type                   0.00000
Soil_Name_lower             0.00000
1_% Total Clay < 0.002      0.00000
1_%Total Silt 0.002-0.05    0.00000
1_% Total Sand 0.05-2       0.00000
2_% Total Clay < 0.002      0.00000
2_%Total Silt 0.002-0.05    0.00000
2_% Total Sand 0.05-2       0.00000
3_% Total Clay < 0.002      0.00000
3_%Total Silt 0.002-0.05    0.00000
3_% Total Sand 0.05-2       0.00000
4_% Total Clay < 0.002      0.00000
4_%Total Silt 0.002-0.05    0.00000
4_% Total Sand 0.05-2       0.00000
5_% Total Clay < 0.002      0.00000
5_%Total Silt 0.002-0.05    0.00000
5_% Total Sand 0.05-2  

 15%|████████████▊                                                                      | 2/13 [00:37<03:24, 18.55s/it]

...

 85%|█████████████████████████████████████████████████████████████████████▍            | 11/13 [04:17<00:55, 27.87s/it]

Elkville
RM                          0.0
Yield (Bu/A)                0.0
Moisture %                  0.0
Lodging %                   0.0
Year                        0.0
Previous Crop               0.0
Previous Pest               0.0
FertN (units)               0.0
Seed Rate (/A)              0.0
Seed Date                   0.0
Harvest Date                0.0
Soil Type                   0.0
Soil_Name_lower             0.0
1_% Total Clay < 0.002      0.0
1_%Total Silt 0.002-0.05    0.0
1_% Total Sand 0.05-2       0.0
2_% Total Clay < 0.002      0.0
2_%Total Silt 0.002-0.05    0.0
2_% Total Sand 0.05-2       0.0
3_% Total Clay < 0.002      0.0
3_%Total Silt 0.002-0.05    0.0
3_% Total Sand 0.05-2       0.0
4_% Total Clay < 0.002      0.0
4_%Total Silt 0.002-0.05    0.0
4_% Total Sand 0.05-2       0.0
5_% Total Clay < 0.002      0.0
5_%Total Silt 0.002-0.05    0.0
5_% Total Sand 0.05-2       0.0
6_% Total Clay < 0.002      0.0
6_%Total Silt 0.002-0.05    0.0
6_% Total Sand 0.05-2       0.0

 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [04:26<00:22, 22.28s/it]

Fenton
RM                          0.0
Yield (Bu/A)                0.0
Moisture %                  0.0
Lodging %                   0.0
Year                        0.0
Previous Crop               0.0
Previous Pest               0.0
FertN (units)               0.0
Seed Rate (/A)              0.0
Seed Date                   0.0
Harvest Date                0.0
Soil Type                   0.0
Soil_Name_lower             0.0
1_% Total Clay < 0.002      0.0
1_%Total Silt 0.002-0.05    0.0
1_% Total Sand 0.05-2       0.0
2_% Total Clay < 0.002      0.0
2_%Total Silt 0.002-0.05    0.0
2_% Total Sand 0.05-2       0.0
3_% Total Clay < 0.002      0.0
3_%Total Silt 0.002-0.05    0.0
3_% Total Sand 0.05-2       0.0
4_% Total Clay < 0.002      0.0
4_%Total Silt 0.002-0.05    0.0
4_% Total Sand 0.05-2       0.0
5_% Total Clay < 0.002      0.0
5_%Total Silt 0.002-0.05    0.0
5_% Total Sand 0.05-2       0.0
6_% Total Clay < 0.002      0.0
6_%Total Silt 0.002-0.05    0.0
6_% Total Sand 0.05-2       0.0
N

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [04:30<00:00, 16.78s/it]


In [355]:
# summary func for town in UIUC
def summary_uiuc(Urbana):
    out1 =  NA_count(Urbana)
    Urbana.groupby('Year')['Seed Date'].value_counts(sort=True)
    Seed_date = Urbana.groupby('Year')['Seed Date'].first().tolist()
    Harvest_date = Urbana.groupby('Year')['Harvest Date'].first().tolist()
    Year = Urbana['Year'].unique().tolist()
    Year.sort()
    plt.figure(figsize=(25,25))
    sns.pairplot(Urbana, diag_kind="kde",hue="Year")
    plt.title('Feature correlation plot for Urbana')
    plt.show()
    plt.figure(figsize=(15,15))
    ax = sns.boxplot(y='Yield (Bu/A)', x="Year", hue="Year",data=Urbana)
    plt.title('Yield for different year in Urbana')
    plt.show()
    return({NA_count: out1, Seed_date:Seed_date,Harvest_date:harvest_date,Year:year})
    
 

In [ ]:
#visualize the correlation matrix

plt.figure(figsize=(40,40))
corr = Urbana.select_dtypes(include=['float64']).corr()
corr
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap,  center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Heated map for feature correlation plot for Urbana')
plt.show()

# head for Goodfield

In [ ]:

#distribution

plt.figure(figsize=(25,25))
sns.pairplot(Goodfield, diag_kind="kde",hue="Year")
plt.title('Feature distribution plot for Goodfield')
plt.show()

In [ ]:
# basic see have the yield varied

plt.figure(figsize=(15,15))
ax = sns.boxplot(y='Yield (Bu/A)', x="Year", hue="Year",data=Goodfield)
plt.title('Yield for different year in Goodfield')
plt.show()


In [ ]:
#visualize the correlation matrix

plt.figure(figsize=(40,40))
corr =Goodfield.select_dtypes(include=['float64']).corr()
corr
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap,  center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Heated map for feature correlation plot for Goodfield')
plt.show()

In [ ]:
corr

In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_GDD(start_date=Seed_date[i] , end_date=Harvest_date[i],city = 'Goodfield' )    
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_PRCP(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city = 'Goodfield')    
plt.legend( Year)
plt.show()

## Check the freq of each towns

In [642]:
col = ['Product','Year','Yield (Bu/A)','Previous Yield (bu/A)','Seed_date', 'Harvest_date','Moisture %','Lodging %', 'Harvest Rate (/A)','Seed Rate (/A)','Seed Space (in)','Soil_Type' ]
Winnebago_df =First_11_17_all.loc[First_11_17_all['Location Town']=='Winnebago',col]
Winnebago_df['Soil_Type'].value_counts()
First_11_17_all['Location Town'].value_counts()

Grand Ridge      642
Sublette         614
Malta            612
Winnebago        610
Mazon            549
Walnut           484
Salem            470
Forsyth          462
Flora            456
Watseka          432
Virden           430
Galva            430
Delavan          428
Macomb           425
Woodstock        423
Du Quoin         422
Towanda          420
Belleville       403
Tuscola          401
Milledgeville    400
Williamsville    399
Vandalia         372
Clayton          372
Bethany          338
Warren           245
Thomasboro       186
Greenville       185
Shumway          183
Pearl City       124
Rossville        122
Paxton           121
Geneseo           92
WALNUT            92
MAZON             92
SUBLETTE          92
GRAND RIDGE       92
WINNEBAGO         92
FLORA             62
WOODSTOCK         62
DELAVAN           62
thomasboro        62
JACKSONVILLE      62
BELLEVILLE        62
SALEM             62
RAMSEY            62
GALVA             62
MACOMB            62
virden       

__Check the type of soil in UIUC and First__

In [643]:
first_soil_list= First_11_17_all['Soil Type'].value_counts().index.values
data_soil_list = np.append(uiuc_soil_list,first_soil_list)

### soil dictionary for UIUC and First data

In [644]:
data_soil_list = np.unique(data_soil_list)
np.unique(data_soil_list)

array(['Beaucoup silty clay loam', 'Belknap silt loam',
       'Bluford silt loam', 'Brenton/Clare silt loam', 'Bryce silty clay',
       'Bryce/Mokena silty clay loam', 'Caseyville silt loam',
       'Cisne clay loam', 'Coffeen silt loam', 'Cowden silty clay loam',
       'Dana silty clay loam', 'Darmstadt silt loam',
       'Darmstat silt loam', 'Drummer silty clay loam',
       'Drummer/Dana/Parr/Raub silty clay loam',
       'Drummer/Flanagan silt loam', 'Drummer/Flanagan silty clay loam',
       'Drummer/Richwood silty clay loam', 'Ebbert silt loam',
       'Edwardsville silt loam', 'Elburn/Flanagan silty clay loam',
       'Elliott/Ashkum silty clay loam', 'Elpaso silty clay loam',
       'Flanagan silt loam', 'Flanagan silty clay loam',
       'Flanagan-Dana silt loam', 'Flanagan-Drummer sandy clay loam',
       'Flanagan/Catlin silt loam', 'Gilford/Ridgeville sandy loam',
       'Herrick silt loam', 'Hoyleton silty clay loam',
       'Hoyleton-Darmstadt silt loam', 'Hoyleton/Ci

In [243]:
First_11_17_all['Harvest Date'].isna().mean()

0.0

In [244]:
#drop all the obs with no heavest date
ind = First_11_17_all['Harvest Date'].isna()
First_11_17_all = First_11_17_all.loc[-ind,:]

In [245]:
First_11_17_all['Harvest Date'].isna().mean()
#here I remove all the rows without NA

0.0

In [246]:
First_11_17_all['Harvest Date']

0         9/17/2018
1         9/17/2018
2         9/17/2018
3         9/17/2018
4         9/17/2018
5         9/17/2018
6         9/17/2018
7         9/17/2018
8         9/17/2018
9         9/17/2018
10        9/17/2018
11        9/17/2018
12        9/17/2018
13        9/17/2018
14        9/17/2018
15        9/17/2018
16        9/17/2018
17        9/17/2018
18        9/17/2018
19        9/17/2018
20        9/17/2018
21        9/17/2018
22        9/17/2018
23        9/17/2018
24        9/17/2018
25        9/17/2018
26        9/17/2018
27        9/17/2018
28        9/17/2018
29        9/17/2018
            ...    
14046     10/7/2013
14047     10/3/2015
14048     10/3/2015
14049     10/3/2015
14050     10/9/2013
14051     10/9/2013
14052     10/9/2013
14053     10/4/2016
14054     10/4/2016
14055    10/29/2017
14056    10/25/2013
14057    10/25/2013
14058    10/25/2013
14059    10/21/2015
14060    10/21/2015
14061    10/21/2015
14062    10/21/2015
14063    10/29/2016
14064    10/29/2016


In [356]:
def get_df_FIRST(Winnebago):   
    
    Winnebago = Winnebago.upper()
    def sp_correct(soil):
        if soil == "muscatine": return("muscatune")
        elif soil == "darmstadt":return("darmstat")
        elif soil == "reddick" : return("redick")
        else: return(soil)
    index = First_11_17_all['Location Town'].apply( lambda x : x.upper())
    
    Winnebago_df =First_11_17_all.loc[index== Winnebago,:]
    Num_col = list(Winnebago_df.select_dtypes([np.number]).columns.values)
    Num_col.extend(['Seed Date','Harvest Date','Soil Type'])
    Winnebago_df = Winnebago_df.loc[:,Num_col]
    print(Winnebago_df.shape)
    def select_name(name):
        if len(name.split("/"))>1:return(name.split("/")[0].split()[0].lower())
        else:
            return(name.split()[0].lower())
        
        
    if isinstance( Winnebago_df['Soil Type'].unique()[0], str):
        Winnebago_df['Soil Type'] =  Winnebago_df['Soil Type'].fillna( Winnebago_df['Soil Type'].unique()[0])
    else:Winnebago_df['Soil Type'] =  Winnebago_df['Soil Type'].fillna( Winnebago_df['Soil Type'].unique()[1])
        

    
    Winnebago_df['Soil_Name_lower'] =  Winnebago_df['Soil Type'].apply(lambda x: select_name(x))
    Winnebago_df['Soil_Name_lower']=Winnebago_df['Soil_Name_lower'].apply(lambda x: sp_correct(x))
    Winnebago_df = pd.merge(Winnebago_df, soil_dic_added, how='left',on = 'Soil_Name_lower')

    
    def get_temperature(CITY):
        ind = Weather_interp['CITY'].apply(lambda x:  x.split('_')[1].lower().capitalize()) == CITY.lower().capitalize()

        temperature =Weather_interp.loc[ind,:]

        temperature = temperature[['DATE','CITY','Interpolated TMIN','Interpolated TMAX']]

        temperature = temperature.dropna()
        temperature = temperature.drop_duplicates()
        temperature.columns = ['DATE', 'CITY', 'TMIN', 'TMAX']
        temperature.head()

        upper_index = (temperature[temperature['TMAX'] > 86].index.values.tolist())
        lower_index_tmin = (temperature[temperature['TMIN'] < 50].index.values.tolist())
        lower_index_tmax = (temperature[temperature['TMAX'] < 50].index.values.tolist())
        temperature.loc[upper_index, 'TMAX'] = 86
        temperature.loc[lower_index_tmin, 'TMIN'] = 50
        temperature.loc[lower_index_tmax, 'TMAX'] = 50
        temperature ['GDD'] = temperature.mean(axis=1) - 50
        temperature['PRCP']  = Weather_interp.loc[ind,'Interpolated PRCP']
        return(temperature)


    temperature = get_temperature(Winnebago)  

    def  std_GDD (x):
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])
        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['GDD'].std()#
        return(tmp)

    def std_PRCP(x ):
        start_date=pd.to_datetime(x[0])
        end_date=pd.to_datetime(x[1])
        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['PRCP'].std()#
        return(tmp)




    def row_cumulative_GDD(x,first = False ,last = False,length =21):
        if first == True:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[0])+timedelta(days=length)
        elif last == True:
            start_date=pd.to_datetime(x[1])-timedelta(days=length)
            end_date=pd.to_datetime(x[1])
        else:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[1])


        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['GDD'].sum()#
        return(tmp)

    def row_cumulative_PRCP(x,first = False ,last = False,length =21):

        if first == True:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[0])+timedelta(days=length)
        elif last == True:
            start_date=pd.to_datetime(x[1])-timedelta(days=length)
            end_date=pd.to_datetime(x[1])
        else:
            start_date=pd.to_datetime(x[0])
            end_date=pd.to_datetime(x[1])

        df = temperature
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        df = df.sort_index()
        df = df.drop_duplicates()
        df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
        tmp = df['PRCP'].sum()#
        return(tmp)
    
    
    Winnebago_df['CGDD'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_GDD,axis=1)
    Winnebago_df['std_GDD'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_GDD,axis=1)
    Winnebago_df['CPRCP']= Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(row_cumulative_PRCP,axis=1)
    Winnebago_df['CGDD_first21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,first = True),axis=1)
    Winnebago_df['std_PRCP']= Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(std_PRCP,axis=1)
    Winnebago_df['CGDD_last21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_GDD(x,last = True),axis=1)
    Winnebago_df['CPRCP_first21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,first = True),axis=1)
    Winnebago_df['CPRCP_last21'] =Winnebago_df.loc[:,['Seed Date', 'Harvest Date']].apply(lambda x : row_cumulative_PRCP(x,last = True),axis=1)

    return(Winnebago_df)

In [357]:
First_city_list= First_11_17_all['Location Town'].apply( lambda x : x.upper()).unique() 
First_city_list 

array(['FORSYTH', 'THOMASBORO', 'TOLUCA', 'TOWANDA', 'WATSEKA', 'TUSCOLA',
       'GRAND RIDGE', 'SUBLETTE', 'MAZON', 'WINNEBAGO', 'BELLEVILLE',
       'RAMSEY', 'MARION', 'FLORA', 'SALEM', 'EFFINGHAM', 'WILLIAMSVILLE',
       'VIRDEN', 'DELAVAN', 'GALVA', 'MACOMB', 'JACKSONVILLE',
       'PEARL CITY', 'MILLEDGEVILLE', 'WOODSTOCK', 'MALTA', 'BETHANY',
       'CLAYTON', 'PAXTON', 'VANDALIA', 'SHUMWAY', 'WALNUT', 'WARREN',
       'GREENVILLE', 'DU QUOIN', 'ROSSVILLE', 'GENESEO', 'MOWEAQUA'],
      dtype=object)

In [358]:
with tqdm(total=len(First_city_list)) as pbar:
    for city in First_city_list : 
 
        print(city)
 
        city_clean = city
        city_clean =city_clean.replace(' ','_')
        city_clean =city_clean.replace('._','_')
        
        vars()[city_clean]  = get_df_FIRST(city)
        vars()[city_clean]  = eval(city_clean).drop('Seed Rate (/A)',axis=1)
        eval(city_clean).to_csv(os.path.join(saving_path,(city_clean+"_first"+'.csv')))
        print((eval(city_clean)).isna().mean())
        print((eval(city_clean)).describe())
        pbar.update(1)  



  0%|                                                                                           | 0/38 [00:00<?, ?it/s]

FORSYTH
(524, 19)
RM                          0.000000
Yield (Bu/A)                0.000000
Moisture %                  0.000000
Lodging %                   0.000000
Stand (x1000)               0.000000
Income Rank                 0.000000
Year                        0.000000
Season                      0.000000
Previous Crop               0.000000
Previous Yield (yrs)        0.645038
Soil_pH                     0.000000
Soil_OM%                    0.000000
Soil_CEC                    0.763359
FertN (units)               0.000000
Seed Space (in)             0.000000
Seed Date                   0.000000
Harvest Date                0.000000
Soil Type                   0.000000
Soil_Name_lower             0.000000
1_% Total Clay < 0.002      0.118321
1_%Total Silt 0.002-0.05    0.118321
1_% Total Sand 0.05-2       0.118321
2_% Total Clay < 0.002      0.118321
2_%Total Silt 0.002-0.05    0.118321
2_% Total Sand 0.05-2       0.118321
3_% Total Clay < 0.002      0.118321
3_%Total Silt 0.002-

  3%|██▏                                                                                | 1/38 [00:21<13:21, 21.66s/it]

THOMASBORO
(248, 19)
RM                          0.00
Yield (Bu/A)                0.00
Moisture %                  0.00
Lodging %                   0.00
Stand (x1000)               0.00
Income Rank                 0.00
Year                        0.00
Season                      0.00
Previous Crop               0.00
Previous Yield (yrs)        0.25
Soil_pH                     0.00
Soil_OM%                    0.00
Soil_CEC                    0.50
FertN (units)               0.00
Seed Space (in)             0.00
Seed Date                   0.00
Harvest Date                0.00
Soil Type                   0.00
Soil_Name_lower             0.00
1_% Total Clay < 0.002      0.00
1_%Total Silt 0.002-0.05    0.00
1_% Total Sand 0.05-2       0.00
2_% Total Clay < 0.002      0.00
2_%Total Silt 0.002-0.05    0.00
2_% Total Sand 0.05-2       0.00
3_% Total Clay < 0.002      0.00
3_%Total Silt 0.002-0.05    0.00
3_% Total Sand 0.05-2       0.00
4_% Total Clay < 0.002      0.00
4_%Total Silt 0.002-0.

  5%|████▎                                                                              | 2/38 [00:32<11:02, 18.39s/it]

...

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 36/38 [08:42<00:18,  9.04s/it]

GENESEO
(92, 19)
RM                          0.000000
Yield (Bu/A)                0.000000
Moisture %                  0.000000
Lodging %                   0.000000
Stand (x1000)               0.000000
Income Rank                 0.000000
Year                        0.000000
Season                      0.000000
Previous Crop               0.000000
Previous Yield (yrs)        1.000000
Soil_pH                     0.000000
Soil_OM%                    0.000000
Soil_CEC                    1.000000
FertN (units)               0.000000
Seed Space (in)             0.000000
Seed Date                   0.000000
Harvest Date                0.000000
Soil Type                   0.000000
Soil_Name_lower             0.000000
1_% Total Clay < 0.002      0.000000
1_%Total Silt 0.002-0.05    0.000000
1_% Total Sand 0.05-2       0.000000
2_% Total Clay < 0.002      0.000000
2_%Total Silt 0.002-0.05    0.000000
2_% Total Sand 0.05-2       0.000000
3_% Total Clay < 0.002      0.000000
3_%Total Silt 0.002-0

 97%|███████████████████████████████████████████████████████████████████████████████▊  | 37/38 [08:45<00:07,  7.40s/it]

MOWEAQUA
(62, 19)
RM                          0.0
Yield (Bu/A)                0.0
Moisture %                  0.0
Lodging %                   0.0
Stand (x1000)               0.0
Income Rank                 0.0
Year                        0.0
Season                      0.0
Previous Crop               0.0
Previous Yield (yrs)        1.0
Soil_pH                     0.0
Soil_OM%                    0.0
Soil_CEC                    1.0
FertN (units)               0.0
Seed Space (in)             0.0
Seed Date                   0.0
Harvest Date                0.0
Soil Type                   0.0
Soil_Name_lower             0.0
1_% Total Clay < 0.002      0.0
1_%Total Silt 0.002-0.05    0.0
1_% Total Sand 0.05-2       0.0
2_% Total Clay < 0.002      0.0
2_%Total Silt 0.002-0.05    0.0
2_% Total Sand 0.05-2       0.0
3_% Total Clay < 0.002      0.0
3_%Total Silt 0.002-0.05    0.0
3_% Total Sand 0.05-2       0.0
4_% Total Clay < 0.002      0.0
4_%Total Silt 0.002-0.05    0.0
4_% Total Sand 0.05-2 

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [08:48<00:00,  5.99s/it]


In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_GDD(start_date=Seed_date[i] , end_date=Harvest_date[i],city = 'Winnebago' )    
plt.legend(Year)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
for i in range(len(Seed_date)):   
    show_cumulative_PRCP(start_date=Seed_date[i] , end_date=Harvest_date[i] ,city = 'Winnebago')    
plt.legend(Year)
plt.show()